# Lab 5: Google Speech Commands

## Imports

In [1]:

import os
from pathlib import Path
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Input, Conv1D, AvgPool1D, MaxPool1D, ZeroPadding1D, BatchNormalization, Flatten, Dense, Activation
from keras.utils.data_utils import get_file
from keras.utils.np_utils import to_categorical
import wave 
import xenocanto



In [2]:
birds = ['Emberiza citrinella','Cuculus canorus','Emberiza cirlus']
dataset_dir = Path('dataset')


## Download, cache and extract birds songs from xeno canto

In [4]:

if not (dataset_dir/'testing_list.txt').exists(): # Assume dataset already downloaded/extracted if testing list is present
    for bird in birds : 
        xenocanto.metadata([bird,"type:song","q:A"])
        xenocanto.metadata([bird,"type:song","q:B"])
        await xenocanto.download([bird,"type:song","q:A"],2)
        await xenocanto.download([bird,"type:song","q:B"],2)

after loading the data : in powershell in repository dataset/audio : 
```ps
 Get-ChildItem  . -Recurse -Filter *.mp3 | ForEach-Object {  $input = $_.Fullname $newname = $_.Fullname.replace(".mp3","")+_03d.mp3; ffmpeg -i $input -f segment -segment_time 10 $newname; rm $input}
 Get-ChildItem  . -Recurse -Filter *.mp3 | ForEach-Object {  $input = $_.Fullname;$newname = $_.Fullname.replace(".mp3",".wav"); ffmpeg -i $input -ac 2 -ar 48000 $newname; rm $input}
  ```

In [9]:


if not (dataset_dir/'testing_list.txt').exists(): # Assume dataset already downloaded/extracted if testing list is present

    CLASSES = os.listdir(dataset_dir/"audio")
    
    for c in CLASSES :
        files = os.listdir(dataset_dir/"audio"/c)
        for file in files:
            if file.endswith('.wav'):
                try:
                    wave.open(str(dataset_dir)+"/audio/"+c+"/"+str(file)) 
                except:
                    os.remove(str(dataset_dir)+"/audio/"+c+"/"+str(file)) 
                
    numOfrec =min([len(os.listdir(dataset_dir/"audio"/c)) for c in CLASSES])
    num_test = int(numOfrec*0.1)
    for c in CLASSES :
        files = os.listdir(dataset_dir/"audio"/c)
        for f in files[numOfrec:] :
            os.remove(dataset_dir/"audio"/c/f)
    os.open(dataset_dir/'testing_list.txt', os.O_CREAT)
    os.open(dataset_dir/'validation_list.txt', os.O_CREAT)
    
    for c in CLASSES :
        i = 0        
        for rec in os.listdir(dataset_dir/'audio'/c):
            if rec.endswith('.wav') :
                if i < num_test :
                    with open(dataset_dir/'testing_list.txt', 'a') as f:
                        f.write(c + '/' + rec +'\n')
                elif i < num_test*2:
                    with open(dataset_dir/'validation_list.txt', 'a') as f:
                        f.write(c + '/' + rec +'\n')
                i=i+1
    

3575
3575
3575
357


## Load raw spoken digits data from Google Speech Commands

In [10]:
# Classes to handle, ordered by label
with (dataset_dir/'testing_list.txt').open() as f:
    testing_list = f.read().splitlines()
CLASSES = os.listdir(dataset_dir/"audio")
x_train = []
y_train = []
x_test = []
y_test = []
audiopath = dataset_dir/'audio'
for recording in audiopath.glob(f'**/*.wav'):
    if not recording.parent.name in CLASSES: # Ignore unused classes
        continue
    
    label = CLASSES.index(recording.parent.name) # Assign class number
    with wave.open(str(recording)) as f: # Read wave file
        data = np.frombuffer(f.readframes(f.getnframes()), dtype=np.int16).copy() # As 16-bit signed integer

    data = data.astype(np.float32) # Convert to 32-bit floating-point
    data.resize((10000, 2)) # Resize to 2s (10kHz) with zero-padding, 1 channel
    if str(recording.relative_to(audiopath)).replace('\\','/') in testing_list: # Assign to test set if file in test list
        x_test.append(data)
        y_test.append(label)
    else:
        x_train.append(data)
        y_train.append(label)

print(f'Loaded {len(x_train)} training samples and {len(x_test)} testing samples')
x_train = np.array(x_train)
y_train = to_categorical(np.array(y_train))
x_test = np.array(x_test)
y_test = to_categorical(np.array(y_test))

Loaded 9654 training samples and 1071 testing samples


## Prepare for inference with fixed-point Q7.9 samples by scaling input data accordingly

In [11]:
FIXED_POINT = 9
x_train /= 2**FIXED_POINT
x_test  /= 2**FIXED_POINT

## Export small dataset (250 random vectors)

In [12]:
perms = np.random.permutation(len(y_test))[0:250]
x_test_250 = x_test[perms]
y_test_250 = y_test[perms]
np.savetxt('x_test_gsc_250.csv', x_test_250.reshape((x_test_250.shape[0], -1)), delimiter=',', fmt='%s')
np.savetxt('y_test_gsc_250.csv', y_test_250, delimiter=',', fmt='%s')

## Build model M5

In [13]:
model = Sequential()

model.add(Input(shape=(10000, 2)))
model.add(Conv1D(filters=8, kernel_size=30, activation='relu',strides=10))
model.add(MaxPool1D(pool_size=10))
model.add(Conv1D(filters=8, kernel_size=3, activation='relu'))
model.add(MaxPool1D(pool_size=4))
model.add(Conv1D(filters=16, kernel_size=3, activation='relu'))
model.add(MaxPool1D(pool_size=4))
model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model.add(MaxPool1D(pool_size=1))
model.add(AvgPool1D())
model.add(Flatten())
model.add(Dense(units=3))
model.add(Activation('softmax')) # SoftMax activation needs to be separate from Dense to remove it later on
# EXPLORE Learning Rate
opt = tf.keras.optimizers.Adam(learning_rate=10e-3)
model.summary()
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 998, 8)            488       
                                                                 
 max_pooling1d (MaxPooling1D  (None, 99, 8)            0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 97, 8)             200       
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 24, 8)            0         
 1D)                                                             
                                                                 
 conv1d_2 (Conv1D)           (None, 22, 16)            400       
                                                                 
 max_pooling1d_2 (MaxPooling  (None, 5, 16)            0

## Train model

In [17]:
model.fit(x_train, y_train, epochs=10, batch_size=100, validation_data=(x_test, y_test))

Epoch 1/10
97/97 [==============================] - 2s 20ms/step - loss: 0.8072 - categorical_accuracy: 0.6192 - val_loss: 0.9520 - val_categorical_accuracy: 0.5462
Epoch 2/10
97/97 [==============================] - 2s 19ms/step - loss: 0.8193 - categorical_accuracy: 0.6041 - val_loss: 0.9010 - val_categorical_accuracy: 0.5696
Epoch 3/10
97/97 [==============================] - 2s 21ms/step - loss: 0.8061 - categorical_accuracy: 0.6223 - val_loss: 0.9060 - val_categorical_accuracy: 0.5808
Epoch 4/10
97/97 [==============================] - 2s 20ms/step - loss: 0.8016 - categorical_accuracy: 0.6191 - val_loss: 0.9738 - val_categorical_accuracy: 0.5612
Epoch 5/10
97/97 [==============================] - 2s 18ms/step - loss: 0.8035 - categorical_accuracy: 0.6260 - val_loss: 0.9485 - val_categorical_accuracy: 0.5668
Epoch 6/10
97/97 [==============================] - 2s 18ms/step - loss: 0.8041 - categorical_accuracy: 0.6184 - val_loss: 0.9033 - val_categorical_accuracy: 0.5910
Epoch 7/10

## Evaluate model on test dataset

In [18]:
model.evaluate(x_test, y_test, verbose=2)
pred_test = model.predict(x_test)
print(tf.math.confusion_matrix(y_test.argmax(axis=1), pred_test.argmax(axis=1)))

34/34 - 0s - loss: 0.9352 - categorical_accuracy: 0.5798 - 133ms/epoch - 4ms/step
34/34 [==============================] - 0s 4ms/step
tf.Tensor(
[[220  48  89]
 [ 73 223  61]
 [142  37 178]], shape=(3, 3), dtype=int32)


In [19]:
model.evaluate(x_test_250, y_test_250, verbose=2)
pred_test_250 = model.predict(x_test_250)
print(tf.math.confusion_matrix(y_test_250.argmax(axis=1), pred_test_250.argmax(axis=1)))

8/8 - 0s - loss: 1.0142 - categorical_accuracy: 0.5560 - 54ms/epoch - 7ms/step
8/8 [==============================] - 0s 5ms/step
tf.Tensor(
[[60 12 15]
 [18 47 16]
 [39 11 32]], shape=(3, 3), dtype=int32)


## Save trained model

In [20]:
model.save('lab_gsc.h5')

## Remove SoftMax layer

In [21]:
model = tf.keras.Model(model.input, model.layers[-2].output, name=model.name)

## Install MicroAI for C inference code generation (kerascnn2c module)

In [22]:
%pip install https://bitbucket.org/edge-team-leat/microai_public/get/6adfbcb347d3.zip#subdirectory=third_party/kerascnn2c_fixed
import kerascnn2c

     ---------------------------------------- 1.9/1.9 MB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
Note: you may need to restart the kernel to use updated packages.


## Generate C code for the trained model with 16-bit fixed-point representation

In [23]:
res = kerascnn2c.Converter(output_path=Path('gsc_output_fixed'),
                           fixed_point=FIXED_POINT, # Number of bits for the fractional part, Q7.9 format
                           number_type='int16_t', # Data type for weights/activations (16 bits quantization)
                           long_number_type='int32_t', # Data type for intermediate results
                           number_min=-(2**15), # Minimum value for 16-bit signed integers
                           number_max=(2**15)-1 # Maximum value for 16-bit signed integers
                          ).convert_model(model)
with open('gsc_model_fixed.h', 'w') as f:
    f.write(res)

———————————————————————————————————————————————————————————————————————————————————————————————————————
Inputs                           | Layer                            | Outputs                         
———————————————————————————————————————————————————————————————————————————————————————————————————————
                                 | input_1                          | conv1d                          
-------------------------------------------------------------------------------------------------------
input_1                          | conv1d                           | max_pooling1d                   
-------------------------------------------------------------------------------------------------------
conv1d                           | max_pooling1d                    | conv1d_1                        
-------------------------------------------------------------------------------------------------------
max_pooling1d                    | conv1d_1                         

## Compile the 16-bit fixed-point C code for x86 and evaluate on small dataset

In [24]:

!g++ -Wall -Wextra -pedantic -Ofast -o gsc_fixed -Igsc_output_fixed/ gsc_output_fixed/model.c main.cpp 


gsc_output_fixed/model.c: In function 'void cnn(const number_t (*)[10000], number_t*)':
gsc_output_fixed/model.c:147:18: warning: left operand of comma operator has no effect [-Wunused-value]
     activations1.average_pooling1d_output,
     ~~~~~~~~~~~~~^~~~~~~~~~~~~~~~~~~~~~~~


In [26]:
!"./gsc_fixed" x_test_gsc_250.csv y_test_gsc_250.csv

Testing accuracy: 0.556
